## 模型持久化文件介绍


### 保存方式一

#### 保存代码：
```
saver.save(sess,'./tmp/my_model/model.ckpt')
```

计算图结构（Meta graph）：
model.ckpt-30001.meta 

最新版本模型和所有版本模型路径：
checkpoint 

model.ckpt文件（Checkpoint file），保存所有的weights,biases,gradients和其他variables的值：

model.ckpt-30001.data-00000-of-00001
model.ckpt-30001.index


```
#恢复模型
with tf.Session() as sess:
    saver = tf.train.import_meta_graph('path/to/model/model.meta')
    new_saver.restore(sess,tf.train.latest_checkpoint('path/to/model/model.ckpt‘))
    
    #加载部分变量
    graph = tf.get_default_graph()
    fc7= graph.get_tensor_by_name('fc7:0')
```
**此种保存形式只能在Tensorflow框架下使用**

### 保存方式二

.pb格式文件

#### 保存代码:
```
from tensorflow.python.framework import graph_util

# convert_variables_to_constants 需要指定output_node_names，list()，可以多个
constant_graph = graph_util.convert_variables_to_constants(sess, sess.graph_def, ['op_to_store'])

    # 测试 OP
    feed_dict = {x: 10, y: 3}
    print(sess.run(op, feed_dict))

    # 写入序列化的 PB 文件
    with tf.gfile.FastGFile(pb_file_path+'model.pb', mode='wb') as f:
        f.write(constant_graph.SerializeToString())
```

#### 恢复模型：

```
#读取google训练好的模型classify_image_graph_def.pb
inception_graph_def_file=os.path.join(LOG_DIR,'classify_image_graph_def.pb')
with tf.Session() as sess:
    #创建一个图来存放训练好的模型
    with tf.gfile.FastGFile(inception_graph_def_file,'rb') as f:
        graph_def=tf.GraphDef()
        graph_def.ParseFromString(f.read())
        tf.import_graph_def(graph_def,name='')
        
    #保存图结构：保存之前将之前保存的events.out.tfevents......iMac文件删除
    #tools.delete_dir_file(LOG_DIR,'iMac')
    
    writer=tf.summary.FileWriter(LOG_DIR,sess.graph)
    writer.close()

```

### [点击下载inception-v3.pd文件](http://download.tensorflow.org/models/image/imagenet/inception-2015-12-05.tgz)
数据解压之后有以下文件：

* classify_image_graph_def.pb
* imagenet_2012_challenge_label_map_proto.pbtxt
* imagenet_synset_to_human_label_map.txt

其中后面两个用于最后的分类显示，相当于网络最后一层softmax得出得分类是一个字符串，然后这个字符串在文件里面查找出对应的human_label.直接用inception-v3分类需要

In [4]:

#现在下载好inception-v3.pd文件，将其读入并保存计算图，
#以便与使用tensorboard查看其结构
#读取google训练好的模型classify_image_graph_def.pb

import tensorflow as tf 

INCEPTION_V3_PD='tmp/inception_v3/classify_image_graph_def.pb'
INCEPTION_V3_GRAPH_DEF='tmp/inception_v3_graph_def'

with tf.Session() as sess:
    #创建一个图来存放训练好的模型
    with tf.gfile.FastGFile(INCEPTION_V3_PD,'rb') as f:
        graph_def=tf.GraphDef()
        graph_def.ParseFromString(f.read())
        tf.import_graph_def(graph_def,name='inception-v3')
    
    writer=tf.summary.FileWriter(INCEPTION_V3_GRAPH_DEF,sess.graph)
    writer.close()


运行后得到文件：

events.out.tfevents.1530084131.c82682dd8be5

执行下面命令便可查看模型结构：

tensorboard --logdir=path/to/events.out.tfevents..._save_dir

输入张量
![](https://upload-images.jianshu.io/upload_images/1271438-12159ef39308d192.png?imageMogr2/auto-orient/strip%7CimageView2/2/w/1240)

![](https://upload-images.jianshu.io/upload_images/1271438-a1fd3f34e2b40aa9.png?imageMogr2/auto-orient/strip%7CimageView2/2/w/1240)

现在将进行的操作便是：
构建一个自己的softmax全连接层，连接到张量pool_3/_reshape:0后面，以此实现迁移学习

In [5]:
#迁移网络结构-向前传播
import tensorflow as tf

DECODE_JEPG_CONTENTS='DecodeJepg/contents:0'
POOL_3_RESHAPE_NAME='pool_3/_reshape:0'

MIDDLE_NODE=500
NUM_CLASS=5

LEARNING_RATE=0.0001
STEPS=300
BATCH=32

"""
inception-v3的输入是图片raw data,对应的y为 [1 0 0 0 0]
"""

def inference_custom(pool3_reshape_tensor):
    
    #输入为pool3_reshape_tensor,shape:[1,2048]
    
    #自定义全连接层一
    with tf.variable_scope('custom_layer_1'):
        cl1_weights=tf.get_variable(
            'weights',
            [2048,MIDDLE_NODE],
            initializer=tf.truncated_normal_initializer(stddev=0.1)
        )
        
        cl1_biases=tf.get_variable(
            'biases',
            [MIDDLE_NODE],
            initializer=tf.constant_initializer(0.1)
        )
    
        cl1=tf.nn.relu(tf.matmul(pool3_reshape_tensor,cl1_weights)+cl1_biases)
    
    #自定义全连接层二
    with tf.variable_scope('custom_layer_2'):
        cl2_weights=tf.get_variable(
            'weights',
            [2048,NUM_CLASS],
            initializer=tf.truncated_normal_initializer(stddev=0.1)
        )
        
        cl2_biases=tf.get_variable(
            'biases',
            [NUM_CLASS],
            initializer=tf.constant_initializer(0.1)
        )
    
        logit=tf.matmul(cl1,cl2_weights)+cl2_biases
    
    return logit

In [ ]:
#迁移网络结构-训练部分

#读取inception-v3.pd
INCEPTION_V3_PD='tmp/inception_v3/classify_image_graph_def.pb'
with tf.gfile(FastGFile(INCEPTION_V3_PD),'rb') as f:
    graph_def=tf.GraphDef()
    graph_def.ParseFromString(f.read())
    
    decode_jepg_contents_tensor,pool_3_reshape_tensor=tf.import_graph_def(
        graph_def,
        return_elements=[DECODE_JEPG_CONTENTS,POOL_3_RESHAPE_NAME])

#输入变量初始化
x=tf.placeholder(
    tf.decode_raw,
    
)

    
with tf.Session() as sess:
    
    
    
    #向前传播
    y=inference_custom(pool_3_reshape_tensor)
    global_step=tf.Variable(0,trainable=False)
    
    cross_entropy=tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y,labels=tf.argmax(y_,1))
    
    
    
    
    
    
    
    
    #载入图片
    for root, dirs, files in os.walk(VERIFY_IMAGE_PATH):
        for f in files:
            image_data=tf.gfile.FastGFile(os.path.join(root,f),'rb').read()
            predixtions=sess.run(softmax_tensor,{'DecodeJpeg/contents:0':image_data})
            predixtions=numpy.squeeze(predixtions)
            
            top_ks=predixtions.argsort()[-5:][::-1]
            node_id=top_ks[0]
            description_string = node_lookup.id_to_description(node_id)
            score = predixtions[node_id]
            print('Pic is {},Identified as {},and the score is:{}'.format(f, description_string, score))
